## Data preparation for student distribution

In [1]:
# packages
import numpy as np
import pandas as pd

# read in raw data file
air_raw = pd.read_csv('air-raw.csv')

# split off city info
cbsa_info = air_raw.iloc[:, 0:2].dropna().set_index('CBSA')
cbsa_info.to_csv('cbsa-info.csv')

# remove city, state from air quality data
air_quality = air_raw.drop(columns = 'Core Based Statistical Area').set_index('CBSA')
air_quality.to_csv('air-quality.csv')

## Part I: Dataset

Merge the city information with the air quality data and tidy the dataset (see notes below). Write a brief description of the data.

In your description, answer the following questions:

- What is a CBSA (the geographic unit of measurement)?
- How many CBSA's are included in the data?
- In how many states and territories do the CBSA's reside?
- In which years were data values recorded?
- How many observations are recorded?
- How many variables are measured?
- Which variables are non-missing most of the time (*i.e.*, in at least 50% of instances)?
- What is PM 2.5 and why is it important?

Please write your description in narrative fashion; _**please do not list answers to the questions above one by one**_. A few brief paragraphs should suffice; please limit your data description to three paragraphs or less.

### Air quality data

*Write your description here.*

## Part II: Descriptive analysis

Focus on the PM2.5 measurements that are non-missing most of the time. Answer each of the following questions in a brief paragraph or two. Your paragraph(s) should indicate both your answer and a description of how you obtained it; _**please do not include codes with your answers**_.

### Has PM 2.5 air pollution improved in the average U.S. city since 2000?

Write your answer here.

### Over time, has PM 2.5 pollution become more variable, less variable, or about the same from city to city?

Write your answer here.

### Which state has seen the greatest improvement over time?

Write your answer here.

### Choose a location with some meaning to you (e.g. hometown, family lives there, took a vacation there, etc.). Was that location in compliance with EPA primary standards as of the most recent measurement?

Write your answer here.

---

# Codes

In [2]:
# packages
import numpy as np
import pandas as pd
import altair as alt

# raw data
air_raw = pd.read_csv('air-quality.csv')
cbsa_info = pd.read_csv('cbsa-info.csv')

## PART I
##########

air_tidy = cbsa_info.merge(
    air_raw, 
    how = 'right', 
    on = 'CBSA'
).melt(
    id_vars = np.append(air_raw.columns[1:4], cbsa_info.columns),
    var_name = 'Year'
).pivot(
    index = np.append(cbsa_info.columns, 'Year'),
    columns = ['Pollutant', 'Trend Statistic'],
    values = 'value'
)

# fix indices
air_tidy.columns = air_tidy.columns.to_flat_index()
air = air_tidy.reset_index()

# split cbsa into city and state
air[['City', 'State']] = air['Core Based Statistical Area'].str.split(', ', 1, expand = True)

# index by cbsa code
air = air.drop(columns = 'Core Based Statistical Area').set_index(['City', 'State']).reset_index().set_index('CBSA')

# print
air.head()

,City,State,Year,"(PM10, 2nd Max)","(PM2.5, Weighted Annual Mean)","(PM2.5, 98th Percentile)","(O3, 4th Max)","(CO, 2nd Max)","(SO2, 99th Percentile)","(NO2, Annual Mean)","(NO2, 98th Percentile)","(Pb, Max 3-Month Average)"
CBSA,,,,,,,,,,,,
10100,Aberdeen,SD,2000,50.0,8.6,23.0,NaN,NaN,NaN,NaN,NaN,NaN
10100,Aberdeen,SD,2001,58.0,8.6,23.0,NaN,NaN,NaN,NaN,NaN,NaN
10100,Aberdeen,SD,2002,59.0,7.9,20.0,NaN,NaN,NaN,NaN,NaN,NaN
10100,Aberdeen,SD,2003,66.0,8.4,21.0,NaN,NaN,NaN,NaN,NaN,NaN
10100,Aberdeen,SD,2004,39.0,8.1,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# there are 351 distinct CBSA's
air.index.unique().size

351

In [4]:
# many CBSAs cross state boundaries
air.State.unique()

array(['SD', 'MI', 'OH', 'GA', 'NY', 'NM', 'PA-NJ', 'PA', 'AK', 'WI',
       'NC', 'NJ', 'GA-SC', 'ME', 'TX', 'CA', 'MD', 'MA', 'LA', 'WA',
       'VT', 'NH-VT', 'MT', 'AL', 'ND', 'IL', 'IN', 'ID', 'MA-NH', 'CO',
       'KY', 'CT', 'FL', 'IA', 'WV', 'SC', 'NC-SC', 'TN-GA', 'WY',
       'IL-IN-WI', 'OH-KY-IN', 'TN-KY', 'MS', 'GA-AL', 'IA-IL', 'DE',
       'MN-WI', 'OR', 'IN-KY', 'ND-MN', 'AZ', 'MD-WV', 'AR', 'WV-KY-OH',
       'WY-ID', 'MO', 'HI', 'MO-KS', 'NH', 'TN-VA', 'TN', 'NV', 'NE',
       'KY-IN', 'OK', 'TN-MS-AR', 'NY-NJ-PA', 'UT', 'NE-IA', 'KY-IL',
       'PA-NJ-DE-MD', 'PR', 'OR-WA', 'RI-MA', 'VA', 'MN', 'MO-IL',
       'MD-DE', 'IN-MI', 'VA-NC', 'DC-VA-MD-WV', 'WV-OH', 'KS', 'VA-WV',
       'MA-CT', 'OH-PA'], dtype=object)

In [5]:
# number of unique state/territory abbreviations
air.State.str.split('-', 4, expand = True).melt().value.unique().size

53

In [6]:
# 20 years of data
air.Year.unique().size

20

In [7]:
# observations and variables
air.shape

(7020, 12)

In [8]:
# only pm2.5 and o3 mostly non-missing
air.isna().mean()

City                             0.000000
State                            0.000000
Year                             0.000000
(PM10, 2nd Max)                  0.706553
(PM2.5, Weighted Annual Mean)    0.390313
(PM2.5, 98th Percentile)         0.390313
(O3, 4th Max)                    0.190883
(CO, 2nd Max)                    0.831909
(SO2, 99th Percentile)           0.746439
(NO2, Annual Mean)               0.746439
(NO2, 98th Percentile)           0.809117
(Pb, Max 3-Month Average)        0.957265
dtype: float64

In [9]:
## PART II
###########

# focus on pm2.5
air_sub = air[air.columns[air.isna().mean() < 0.5]].iloc[:, 0:5]
air_sub.columns = np.append(air_sub.columns[0:3], ['pm_mean', 'pm_98th'])
air_sub.head()

# annual means are decreasing, variance doesn't change much
annual_means = air_sub.groupby('Year').mean()
annual_means_long = annual_means.reset_index().melt(id_vars = 'Year')
annual_sds = air_sub.groupby('Year').std()
annual_sds_long = annual_sds.reset_index().melt(id_vars = 'Year')
annual_means_long['lwr'] = annual_means_long['value'] - annual_sds_long['value']
annual_means_long['upr'] = annual_means_long['value'] + annual_sds_long['value']

trend = alt.Chart(annual_means_long).encode(
    x = 'Year',
    y = 'value',
    color = 'variable'
).mark_line()

band = alt.Chart(annual_means_long).encode(
    x = 'Year',
    y = 'lwr',
    y2 = 'upr',
    color = 'variable'
).mark_errorband()

fig1 = trend + band
fig1

alt.LayerChart(...)

In [10]:
# summarize by state, counting overlapping regions in each state covered
state_expansion = air_sub.State.str.split('-', 4, expand = True)
state_expansion.columns = ['state1', 'state2', 'state3', 'state4']
state_worst = state_expansion.merge(
    air_sub,
    how = 'left',
    on = 'CBSA'
).drop(
    columns = 'State'
).melt(
    id_vars = ['City', 'Year', 'pm_mean', 'pm_98th'],
    value_name = 'State'
).drop(
    columns = 'variable'
).dropna().drop(
    columns = 'City'
).groupby(
    ['State', 'Year']
).aggregate('max')

# 2019 - 2000 differences by state
state_diffs = state_worst.loc[:, ['2000', '2019'], :].diff().loc[:, '2019', :]

In [11]:
# best improvement, by pm mean
state_diffs.sort_values('pm_mean').iloc[0:5, :].pm_mean

State
OH   -11.3
CA   -11.0
AL   -10.8
VA    -9.3
GA    -9.3
Name: pm_mean, dtype: float64

In [12]:
# best improvement, by pm 98th percentile
state_diffs.sort_values('pm_98th').iloc[0:5, :].pm_98th

State
CA   -57.0
MT   -42.0
AL   -32.0
AK   -29.0
UT   -24.0
Name: pm_98th, dtype: float64

In [13]:
# check sb
air_sub[air_sub.State == 'CA'].City.unique()
air_sub[(air_sub.City == 'Santa Maria-Santa Barbara') & (air_sub.Year == '2019')]

,City,State,Year,pm_mean,pm_98th
CBSA,,,,,
42200,Santa Maria-Santa Barbara,CA,2019,4.8,12.0


In [14]:
# which city had that extreme difference in 98th percentile?
air_sub.set_index(
    ['City', 'Year']
).drop(
    columns = 'State'
).loc[:, ['2000', '2019'], :].sort_index(
    level = 0
).diff().loc[:, '2019', :].sort_values(
    'pm_98th'
).iloc[0, :]

pm_mean   -11.0
pm_98th   -57.0
Name: Visalia-Porterville, dtype: float64

In [15]:
air_sub[air_sub.City == 'Visalia-Porterville']

,City,State,Year,pm_mean,pm_98th
CBSA,,,,,
47300,Visalia-Porterville,CA,2000,23.9,103.0
47300,Visalia-Porterville,CA,2001,22.5,96.0
47300,Visalia-Porterville,CA,2002,23.2,70.0
47300,Visalia-Porterville,CA,2003,18.2,47.0
47300,Visalia-Porterville,CA,2004,17.0,54.0
47300,Visalia-Porterville,CA,2005,18.8,65.0
47300,Visalia-Porterville,CA,2006,18.8,50.0
47300,Visalia-Porterville,CA,2007,20.4,60.0
47300,Visalia-Porterville,CA,2008,19.8,62.0


In [16]:
## EC
######

# could potentially use pm to predict SO2
air.corr()

,"(PM10, 2nd Max)","(PM2.5, Weighted Annual Mean)","(PM2.5, 98th Percentile)","(O3, 4th Max)","(CO, 2nd Max)","(SO2, 99th Percentile)","(NO2, Annual Mean)","(NO2, 98th Percentile)","(Pb, Max 3-Month Average)"
"(PM10, 2nd Max)",1.000000,-0.045930,0.079598,0.099132,0.344990,0.020013,0.415780,0.391540,-0.251197
"(PM2.5, Weighted Annual Mean)",-0.045930,1.000000,0.738820,0.680921,0.217951,0.637939,0.431196,0.562625,0.027223
"(PM2.5, 98th Percentile)",0.079598,0.738820,1.000000,0.558023,0.282994,0.473462,0.423815,0.461943,-0.066222
"(O3, 4th Max)",0.099132,0.680921,0.558023,1.000000,0.329461,0.348519,0.520512,0.572854,0.040126
"(CO, 2nd Max)",0.344990,0.217951,0.282994,0.329461,1.000000,0.314266,0.609102,0.566425,0.029316
"(SO2, 99th Percentile)",0.020013,0.637939,0.473462,0.348519,0.314266,1.000000,0.278752,0.274508,0.209959
"(NO2, Annual Mean)",0.415780,0.431196,0.423815,0.520512,0.609102,0.278752,1.000000,0.871111,-0.172479
"(NO2, 98th Percentile)",0.391540,0.562625,0.461943,0.572854,0.566425,0.274508,0.871111,1.000000,-0.143588
"(Pb, Max 3-Month Average)",-0.251197,0.027223,-0.066222,0.040126,0.029316,0.209959,-0.172479,-0.143588,1.000000
